# Get all the data from users on Admin

## Scraping on a site with login

In [ ]:
# Prepare a function to read the configuration file with the confidential data...
# ...because the website is an private website on an Intranet, the production file is config.yaml.
# In GitHub there is only a config.yaml.example with dummy data

import yaml

__config = None

def config():

    global __config

    if not __config:
        with open('./scraper_ETL/extract/config.yaml', mode='r') as f:
            config = yaml.safe_load(f)

    return config


In [ ]:
# Get the url on the config file
host = config()['websites']['admin_users']['url']
#host

In [ ]:
#Import selenium
#Import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager

# Setup driver
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
#driver = webdriver.Chrome(executable_path='./driver/chromedriver', options=options)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Get the site (driver)
driver.get(host)


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.support.select import Select
from webdriver_manager.firefox import GeckoDriverManager

driver = webdriver.Firefox(
    service=FirefoxService(
        GeckoDriverManager().install(),
    ),
)


In [ ]:
driver.get(host)

In [ ]:
import argparse
import csv
import datetime
import logging
import time

# import chromedriver_binary
# Import config file
from common import config

parser = argparse.ArgumentParser()

# Build the list of choices
websites_choices = list(config()['websites'].keys())
parser.add_argument('website',
                    help='Argument: The website that you want to scrape',
                    type=str,
                    choices=websites_choices)

# Parser the arguments
args = parser.parse_args()
_accounts_scraper(args.website)

In [ ]:
driver.find_elements_by_xpath('//input[@name="pwd"]')

In [ ]:
# Get the credential data
user = config()['websites']['admin_users']['user']
password = config()['websites']['admin_users']['password']

bnt_aceptar = config()['websites']['admin_users']['labels']['page01']['bnt_aceptar']


In [ ]:
# Login
driver.find_element("xpath", "//input[@name='usuario']").send_keys(user)
driver.find_element("xpath", "//input[@name='pwd']").send_keys(password)

# Click to login... the element is an input without name and an alt tag defined on config.yaml: 
    # <input type="image" name src="bt_aceptar.gif" alt="Entar al Sistema">
driver.find_element("xpath", "//input[@alt='" + bnt_aceptar + "']").click()

### Successful login!

## Navigate to the important data

In [ ]:
# Locate the menu option
# <a href="javascript:catchSelection(4000);">Control de Usuarios</a>
menu_usuarios = config()['websites']['admin_users']['labels']['page02']['menu_usuarios']

driver.find_element("xpath", "//a[text()='" + menu_usuarios + "']")


In [ ]:
driver.find_element("xpath", "//a[text()='" + menu_usuarios + "']").click()

In [ ]:
# Now, the next element is a select menu, so we need to import Select from Selenium
from selenium.webdriver.support.select import Select


In [ ]:
# Inspect the select and print the options

# First we get the name of the select menu
name_select_menu = config()['websites']['admin_users']['labels']['page03']['name_select_menu']

test = Select(driver.find_element("name", name_select_menu))
# test.select_by_value("Consulta de Usuarios")
print(test.options[0].text)
print(test.options[1].text)
print(test.options[2].text)
print(test.options[3].text)
print(test.options[4].text)

In [ ]:
# Select the correct option
test = Select(driver.find_element("name", name_select_menu))
option_select_menu = config()['websites']['admin_users']['labels']['page03']['option_select_menu']
test.select_by_value(option_select_menu)

In [ ]:
# Now, we had selected the option on the menu, click on button "Aceptar"
bnt_entra = config()["websites"]["admin_users"]["labels"]["page03"]["bnt_entra"]
driver.find_element("xpath", "//img[@alt='" + bnt_entra + "']").click()

In [ ]:
# Query all the accounts

# <img src="bt_consulta.gif" alt="Consultar" ... >
# Click on button
bnt_cons = config()["websites"]["admin_users"]["labels"]["page04"]["bnt_cons"]
driver.find_element("xpath", "//img[@alt='" + bnt_cons + "']").click()

## Get the data

In [ ]:
# Inspect pagination

# Find element
# rows = driver.find_element_by_xpath('//table[@class="txt"]').get_attribute('cellspacing')
# rows = driver.find_elements_by_xpath('//table[@class="txt"]/tbody/tr/td/table/tbody/tr') //26 rows (max 13 rows)
# Select rows of data without the titles row (not @valign) and even rows // 65 data

class_table = config()['websites']['admin_users']['labels']['page05']['class_table']
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
even_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[@class="txtb"]')

len(even_rows)

In [ ]:
# Select rows of data without the titles row (not @valign) and even rows // 60 data (max 12 rows)
odd_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[@class="txt"]')

len(odd_rows)


In [ ]:
# Testing with the first and second rows
i = 0
j = 0

curp = even_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = even_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = even_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = even_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = even_rows[i].text
print('Grupo:', grupo)
i = i + 1

curp = odd_rows[j].text
print('CURP:', curp)
j = j + 1
matricula = odd_rows[j].text
print('Matrícula:', matricula)
j = j + 1
nombre = odd_rows[j].text
print('Nombre:', nombre)
j = j + 1
cuenta = odd_rows[j].text
print('Cuenta:', cuenta)
j = j + 1
grupo = odd_rows[j].text
print('Grupo:', grupo)
j = j + 1

In [ ]:
# Or, even better, select all the data in the same object with:

class_table = config()['websites']['admin_users']['labels']['page05']['class_table']
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']

all_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "' + class_td + '")]')

len(all_rows)

# //table[@class="txt"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "txt")]



In [ ]:
# Testing with the first and second rows
i = 0

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1
print("")

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1


In [ ]:
# Testing with another row
i=120

print("Last Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1

### Successful scrapper on page data!

## Change to the next page

In [ ]:
# Count the pages...
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
class_a = config()['websites']['admin_users']['labels']['page05']['class_a']
pages = driver.find_element_by_xpath('//td[@class="' + class_td + '"]').find_elements_by_xpath('//a[@class="' + class_a + '"]')
pages

In [ ]:
initial_page = pages[0]
initial_page.text

In [ ]:
final_page = pages[1]
final_page.text

In [ ]:

    # Get the url of the website (parameter)
    pagination01 = config()['websites']['admin_users']['labels']['page01']['url_pagination01']
    pagination02 = config()['websites']['admin_users']['labels']['page01']['url_pagination02']
    pagination = pagination01 + "0" + pagination02 + "1"
    pagination

In [ ]:
# Change to page 2 and so on...
# ...get the link to next page

initial_page.get_attribute('href')

In [ ]:
# Change to next page

initial_page.click()


In [ ]:
# Count the pages...
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
class_a = config()['websites']['admin_users']['labels']['page05']['class_a']
pages = driver.find_element_by_xpath('//td[@class="' + class_td + '"]').find_elements_by_xpath('//a[@class="' + class_a + '"]')
len(pages)


In [ ]:
print(pages[0].text)
print(pages[1].text)
print(pages[2].text)

In [ ]:
pages[1].click()

In [ ]:
# Count the pages...
class_td = config()['websites']['admin_users']['labels']['page05']['class_td']
class_a = config()['websites']['admin_users']['labels']['page05']['class_a']
pages = driver.find_element_by_xpath('//td[@class="' + class_td + '"]').find_elements_by_xpath('//a[@class="' + class_a + '"]')
len(pages)

In [ ]:
print(pages[0].text)
print(pages[1].text)
print(pages[2].text)
print(pages[3].text)

In [ ]:
pages[2].click()

In [ ]:
# Again, select all the data in the same object with:

all_rows = driver.find_elements_by_xpath('//table[@class="' + class_table + '"]/tbody/tr/td/table/tbody/tr[not(@valign)]/td[contains(@class, "' + class_td + '")]')

len(all_rows)

In [ ]:
# Testing with the first and second rows
i = 0

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1
print("")

print("Record #", int(i / 5 + 1))
curp = all_rows[i].text
print('CURP:', curp)
i = i + 1
matricula = all_rows[i].text
print('Matrícula:', matricula)
i = i + 1
nombre = all_rows[i].text
print('Nombre:', nombre)
i = i + 1
cuenta = all_rows[i].text
print('Cuenta:', cuenta)
i = i + 1
grupo = all_rows[i].text
print('Grupo:', grupo)
i = i + 1

### Get each account details

In [ ]:
lista_cuentas = []

In [ ]:
for account in lista_cuentas:
    print(account)
    
    

In [ ]:
driver.find_element("name", "id_login").send_keys("")
base_config = config()["websites"]["admin_users"]["labels"]
bnt_cons = base_config["page04"]["bnt_cons"]
driver.find_element(
    "xpath",
    '//img[@alt="' + bnt_cons + '"]',
).click()

In [ ]:
# Repetir lo siguiente para cada registro:

# Select Consulta "navigate_to_data2"
base_config = config()["websites"]["admin_users"]["labels"]
name_select_menu = base_config["page03"]["name_select_menu"]

# ... Select the correct option
test = Select(driver.find_element("name", name_select_menu))
option_select_menu = base_config["page03"]["option_select_menu"]
test.select_by_value(option_select_menu)

# Now, we had selected the option on the menu, click on button "Aceptar"
bnt_entra = base_config["page03"]["bnt_entra"]
driver.find_element(
    "xpath",
    '//img[@alt="' + bnt_entra + '"]',
).click()

In [ ]:
driver.find_element("name", "id_login").send_keys("RMTR099")

In [ ]:
base_config = config()["websites"]["admin_users"]["labels"]
bnt_cons = base_config["page04"]["bnt_cons"]
driver.find_element(
    "xpath",
    '//img[@alt="' + bnt_cons + '"]',
).click()

In [ ]:
input_curp = driver.find_element("name", "curp")
curp = input_curp.get_attribute("value")
curp

In [ ]:
matricula = driver.find_element(
    "name",
    "matricula",
).get_attribute("value")
matricula

In [ ]:
area = Select(
    driver.find_element("name", "area"),
).first_selected_option.text

genero = Select(
    driver.find_element("name", "genero"),
).first_selected_option.text

nombre = driver.find_element(
    "name",
    "nombre",
).get_attribute("value")

apellido_pat = driver.find_element(
    "name",
    "apellidoPat",
).get_attribute("value")

apellido_mat = driver.find_element(
    "name",
    "apellidoMat",
).get_attribute("value")

telefono = driver.find_element(
    "name",
    "telefono",
).get_attribute("value")

email = driver.find_element(
    "name",
    "email",
).get_attribute("value")

grupo = Select(
    driver.find_element(
        "name",
        "idGrupo",
    )
).first_selected_option.text

usrnametxt = driver.find_element(
    "name",
    "usrnametxt",
).get_attribute("value")

detalle_cuenta = dict(
    curp=curp,
    matricula=matricula,
    area=area,
    genero=genero,
    nombre=nombre,
    apellido_p=apellido_pat,
    apellido_m=apellido_mat,
    telefono=telefono,
    email=email,
    grupo=grupo,
    usuario=usrnametxt,
)

detalle_cuenta

In [ ]:
# Click on "Regresar"

driver.find_element(
    "xpath",
    "/html/body/form/table[3]/tbody/tr[2]/td/img[1]",
).click()

# Repetir este ciclo

### It works!